In [1]:
import torch

from torch import nn
from torch.autograd import Variable

In [2]:
input_parameters = 5
layer_size = 5
layer_count = 3
classification_size = 1
learning_rate = 0.001

In [3]:
graph_stat = Variable(torch.ones(input_parameters, layer_size))

In [4]:
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        self.linear0 = torch.nn.Linear(layer_size, layer_size)
        self.linear1 = torch.nn.Linear(layer_size, layer_size)
        self.linear2 = torch.nn.Linear(layer_size, classification_size)

    def forward(self, x):
        x = self.linear0(x)
        x = self.linear1(x)
        x = self.linear2(x)
        return x

In [5]:
model = NeuralNet()

result = model(graph_stat)
result

tensor([[0.2851],
        [0.2851],
        [0.2851],
        [0.2851],
        [0.2851]], grad_fn=<AddmmBackward0>)

In [6]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [44]:
import pandas
from pandas import DataFrame


def load_dataset(filename: str) -> tuple[list, list]:
    df = pandas.read_csv(filename)
    df.reset_index()
    return [torch.Tensor(v) for v in df.drop('label', axis=1).to_numpy()[:, 1:]], \
           [torch.Tensor([v]) for v in df['label'].to_numpy()]

def f(i, j) -> list[float]:
    return i * 10 + j * j

# dataset = [torch.Tensor([f(i, j) for j in range(input_parameters)]) + torch.randn(input_parameters) for i in range(samples)]
# dataset_labels = [torch.Tensor([i % 3]) for i in range(samples)]
dataset, dataset_labels = load_dataset('dataset.csv')
print(dataset)
print(dataset_labels)

[tensor([36.,  1., 24.,  1., 22.]), tensor([150.,   1.,  60.,   1.,  42.]), tensor([129.,   1.,  42.,   1.,  48.]), tensor([131.,   1.,  40.,   1.,  52.]), tensor([198.,   1.,  64.,   1.,  40.]), tensor([129.,   1.,  42.,   1.,  58.]), tensor([139.,   1.,  46.,   1.,  64.]), tensor([39.,  1., 27.,  1., 25.]), tensor([141.,   1.,  42.,   1.,  59.]), tensor([135.,   1.,  43.,   1.,  58.])]
[tensor([0.]), tensor([0.]), tensor([1.]), tensor([0.]), tensor([2.]), tensor([0.]), tensor([0.]), tensor([0.]), tensor([0.]), tensor([0.])]


In [45]:
bound = int(len(dataset) * 0.8)
train = list(zip(dataset[:bound], dataset_labels[:bound]))
test = list(zip(dataset[bound:], dataset_labels[bound:]))

In [46]:
num_epochs = 10
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Train the model
total_step = len(train)
for epoch in range(num_epochs):
    for i, (image, label) in enumerate(train):
        # Move tensors to the configured device
        image = image.to(device)
        label = label.to(device)

        # Forward pass
        outputs = model(image)
        loss = criterion(outputs, label)

        # Backpropagation and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                   .format(epoch + 1, num_epochs, i + 1, total_step, loss.item()))

In [47]:
# Test the model
# In the test phase, don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct = 0
    total = 0
    for image, label in test:
        image = image.to(device)
        label = label.to(device)
        outputs = model(image)
        _, predicted = torch.max(outputs.data, 0)
        total += label.size(0)
        correct += (predicted == label).item()

    print(f'Accuracy of the network on the {len(test)} test images: {100 * correct / total} %')

Accuracy of the network on the 2 test images: 100.0 %
